# Deliverables:

- Submit a single zip-compressed file that has the name: YourLastName_Exercise_5 that has the following files:

 1. Your **PDF document** that has your Source code and output
 2. Your **ipynb script** that has your Source code and output


# Objectives:

In this exercise, you will:

 - Construct hierarchical indexes
 - Select and group data to create pivot-tables




# Submission Formats :

Create a folder or directory with all supplementary files with your last name at the beginning of the folder name, compress that folder with zip compression, and post the zip-archived folder under the assignment link in Canvas. The following files should be included in an archive folder/directory that is uploaded as a single zip-compressed file. (Use zip, not StuffIt or any 7z or any other compression method.)


1. Complete IPYNB script that has the source code in Python used to access and analyze the data. The code should be submitted as an IPYNB script that can be be loaded and run in Jupyter Notebook for Python
2. Output from the program, such as console listing/logs, text files, and graphics output for visualizations. If you use the Data Science Computing Cluster or School of Professional Studies database servers or systems, include Linux logs of your sessions as plain text files. Linux logs may be generated by using the script process at the beginning of your session, as demonstrated in tutorial handouts for the DSCC servers.
3. List file names and descriptions of files in the zip-compressed folder/directory.


Formatting Python Code
When programming in Python, refer to Kenneth Reitz’ PEP 8: The Style Guide for Python Code:
http://pep8.org/ (Links to an external site.)Links to an external site.
There is the Google style guide for Python at
https://google.github.io/styleguide/pyguide.html (Links to an external site.)Links to an external site.
Comment often and in detail.


# Specifications and Requirements


We're going to use the XYZ data again  to construct hierarchical indexes and select, modify, group, and reshape data in a wide variety of ways.  The data we want here, which we'll call xyzcustnew, are as follows:

In [1]:
import pandas as pd  # panda's nickname is pd
import numpy as np  # numpy as np
from pandas import DataFrame, Series, Categorical 
from sqlalchemy import create_engine

In [2]:
engine=create_engine('sqlite:///xyz.db')	    # the db is in my current working directory

In [3]:
xyzcustnew=pd.read_sql_table('xyzcust',engine)
xyzcustnew.head()

,index,ACCTNO,ZIP,ZIP4,LTD_SALES,LTD_TRANSACTIONS,YTD_SALES_2009,YTD_TRANSACTIONS_2009,CHANNEL_ACQUISITION,BUYER_STATUS,ZIP9_SUPERCODE
0,0,WDQQLLDQL,60084,5016,90.0,1,0.0,0,IB,INACTIVE,600845016
1,1,WQWAYHYLA,60091,1750,4227.0,9,1263.0,3,RT,ACTIVE,600911750
2,2,GSHAPLHAW,60067,900,420.0,3,129.0,1,RT,ACTIVE,600670900
3,3,PGGYDYWAD,60068,3838,6552.0,6,0.0,0,RT,INACTIVE,600683838
4,4,LWPSGPLLS,60090,3932,189.0,3,72.0,1,RT,ACTIVE,600903932


In [4]:
print((xyzcustnew.YTD_SALES_2009[xyzcustnew.BUYER_STATUS=="INACTIVE"]>0).sum())
print((xyzcustnew.YTD_SALES_2009[xyzcustnew.BUYER_STATUS=="LAPSED"]>0).sum())
xyzcustnew.BUYER_STATUS.unique()

0
0


array(['INACTIVE', 'ACTIVE', 'LAPSED'], dtype=object)

In [5]:
# Lets define a cutoff value for what we will consider to be the heavy customers 

heavyCut = xyzcustnew.YTD_SALES_2009[xyzcustnew.BUYER_STATUS=="ACTIVE"].quantile(0.67)
heavyCut

423.0

In [6]:
heavyCat=Categorical(np.where(xyzcustnew.YTD_SALES_2009>heavyCut,1,0))
heavyCat.describe()

,counts,freqs
categories,,
0,25795,0.854733
1,4384,0.145267


In [7]:
heavyCat

[0, 1, 0, 0, 0, ..., 0, 0, 0, 0, 1]
Length: 30179
Categories (2, int64): [0, 1]

In [8]:
heavyCat.rename_categories(['regular','heavy'],inplace=True)
heavyCat

[regular, heavy, regular, regular, regular, ..., regular, regular, regular, regular, heavy]
Length: 30179
Categories (2, object): [regular, heavy]

In [9]:
heavyCat.describe()

,counts,freqs
categories,,
regular,25795,0.854733
heavy,4384,0.145267


In [10]:
heavyCat[:10]

[regular, heavy, regular, regular, regular, regular, heavy, regular, regular, regular]
Categories (2, object): [regular, heavy]

In [11]:
xyzcustnew['heavyCat']=heavyCat

In [12]:
buyerType=pd.get_dummies(heavyCat)
print(type(buyerType), buyerType.shape)

<class 'pandas.core.frame.DataFrame'> (30179, 2)


In [13]:
buyerType[:3]

,regular,heavy
0,1,0
1,0,1
2,1,0


In [14]:
xyzcustnew['typeReg']=buyerType['regular']
xyzcustnew['typeHeavy']=buyerType['heavy']

In [15]:
xyzcustnew.columns

Index(['index', 'ACCTNO', 'ZIP', 'ZIP4', 'LTD_SALES', 'LTD_TRANSACTIONS',
       'YTD_SALES_2009', 'YTD_TRANSACTIONS_2009', 'CHANNEL_ACQUISITION',
       'BUYER_STATUS', 'ZIP9_SUPERCODE', 'heavyCat', 'typeReg', 'typeHeavy'],
      dtype='object')

In [16]:

xyztrans=pd.read_sql('xyztrans', engine)

trandate=xyztrans.TRANDATE	# should be a Series

daystr=trandate.str[0:2]		# two digit date numbers slice

mostr=trandate.str[2:5]		# the three letter month abbreviations

yearstr=trandate.str[5:]		# four digit years


In [17]:
type(mostr)

pandas.core.series.Series

In [18]:
#create a dictionary for the months
monums={'JAN':'1', 'FEB':'2', 'MAR':'3', 'APR':'4', 'MAY':'5', 'JUN':'6', 'JUL':'7', 'AUG':'8', 'SEP':'9', 'OCT':'10', 'NOV':'11','DEC':'12'}
#month
monos=mostr.map(monums)	# do a dict lookup for each value of mostr

transtr=yearstr+'-'+monos+'-'+daystr
type(transtr)

pandas.core.series.Series

In [19]:
trDateTime=pd.to_datetime(transtr)
type(trDateTime)

pandas.core.series.Series

In [20]:
trCounts=trDateTime.value_counts()
trCounts

2009-12-19    877
2009-12-21    836
2009-12-12    782
2009-12-23    765
2009-12-20    744
2009-12-22    717
2009-12-18    708
2009-12-14    615
2009-12-15    599
2009-12-16    571
2009-12-11    568
2009-11-21    561
2009-12-13    542
2009-11-22    507
2009-12-10    504
2009-12-04    488
2009-11-25    451
2009-12-24    425
2009-11-23    421
2009-11-27    419
2009-11-24    412
2009-04-10    404
2009-11-28    402
2009-11-14    402
2009-12-09    401
2009-05-09    398
2009-12-08    397
2009-11-07    394
2009-12-07    372
2009-01-17    372
             ... 
2009-02-26     47
2009-04-30     47
2009-02-21     39
2009-03-01     36
2009-03-10     33
2009-06-14     29
2009-11-26     26
2009-03-16     24
2009-07-14     24
2009-06-15     24
2009-08-30     23
2009-11-04     23
2009-02-25     22
2009-02-02     22
2009-10-17     21
2009-03-08     20
2009-03-27     19
2009-02-03     19
2009-02-01     18
2009-03-11     17
2009-02-07     16
2009-07-02     15
2009-06-13     14
2009-07-16     14
2009-12-25

In [21]:
newIndex=pd.date_range(trCounts.index.min(),trCounts.index.max())

trCountsChrono=trCounts.reindex(index=newIndex)
newIndex

DatetimeIndex(['2009-01-01', '2009-01-02', '2009-01-03', '2009-01-04',
               '2009-01-05', '2009-01-06', '2009-01-07', '2009-01-08',
               '2009-01-09', '2009-01-10',
               ...
               '2009-12-22', '2009-12-23', '2009-12-24', '2009-12-25',
               '2009-12-26', '2009-12-27', '2009-12-28', '2009-12-29',
               '2009-12-30', '2009-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

In [22]:
print(trCountsChrono.head())

2009-01-01    176
2009-01-02    305
2009-01-03    365
2009-01-04    231
2009-01-05    144
Freq: D, Name: TRANDATE, dtype: int64


In [23]:
trDF=DataFrame()                

In [24]:
trDF['date'] = trCountsChrono.index
trDF['transactions'] = trCountsChrono.values


In [25]:
trDF.columns

Index(['date', 'transactions'], dtype='object')

In [26]:
trDF.head()

,date,transactions
0,2009-01-01,176
1,2009-01-02,305
2,2009-01-03,365
3,2009-01-04,231
4,2009-01-05,144


In [27]:
trDF.dtypes

date            datetime64[ns]
transactions             int64
dtype: object

In [28]:
trMed=trDF.transactions.median()		# here's the median

In [29]:
heavyLight=lambda x  : x >= trMed and 'heavy' or 'light'  # an example anon function

In [30]:
trDF.head(4)

,date,transactions
0,2009-01-01,176
1,2009-01-02,305
2,2009-01-03,365
3,2009-01-04,231


In [31]:
trDF['vol']=trDF.transactions.map(heavyLight)	# 'vol' is the heavy/light column

In [32]:
trDF.head(4)

,date,transactions,vol
0,2009-01-01,176,heavy
1,2009-01-02,305,heavy
2,2009-01-03,365,heavy
3,2009-01-04,231,heavy


In [33]:
trDF['monum']=trDF.date.dt.month	# .dt is the datetime accessor
trDF.head(4)

,date,transactions,vol,monum
0,2009-01-01,176,heavy,1
1,2009-01-02,305,heavy,1
2,2009-01-03,365,heavy,1
3,2009-01-04,231,heavy,1


In [34]:
trDFnd=trDF.drop('date',axis=1)	# axis=1 means here a column is selected to drop

In [35]:
trDFnd.head(4)

,transactions,vol,monum
0,176,heavy,1
1,305,heavy,1
2,365,heavy,1
3,231,heavy,1


In [36]:
trDFgrouped=trDFnd.groupby(['vol','monum']).sum()
trDFgrouped

transactions
vol   monum              
heavy 1              5255
      2               761
      3              1130
      4              2327
      5              2172
      6              2878
      7              4440
      8              1682
      9              1921
      10             2109
      11             8402
      12            13168
light 1               572
      2              1625
      3              1664
      4              1727
      5              2076
      6              1495
      7               564
      8              1938
      9              1942
      10             2241
      11               49
      12              257

In [37]:
trDFgrouped=trDFnd.groupby(['monum','vol']).sum()
trDFgrouped

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130
      light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495
7     heavy          4440
      light           564
8     heavy          1682
      light          1938
9     heavy          1921
      light          1942
10    heavy          2109
      light          2241
11    heavy          8402
      light            49
12    heavy         13168
      light           257

In [38]:
trDFgrouped.index.levels

FrozenList([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ['heavy', 'light']])

In [39]:
trDFgrouped.index

MultiIndex(levels=[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ['heavy', 'light']],
           codes=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['monum', 'vol'])

In [40]:
trDFgrouped.head(4)

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625

In [41]:
trDFgrouped.loc[11,'heavy']

transactions    8402
Name: (11, heavy), dtype: int64

In [42]:

trDFgrouped.loc[list(range(1,7))]

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130
      light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495

In [43]:
trDFgrouped.iloc[0:6]		# .iloc here, but .loc above.

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130
      light          1664

In [44]:
trDFgrouped[(3,'light'):(7,'heavy')]

transactions
monum vol                
3     light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495
7     heavy          4440

In [45]:
trDFgrouped[(3,'light'):6]

transactions
monum vol                
3     light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495

In [46]:
trDFgrouped.xs('light',level='vol')

,transactions
monum,
1,572
2,1625
3,1664
4,1727
5,2076
6,1495
7,564
8,1938
9,1942


In [47]:
trDFgrouped.xs('light',level='vol').T		# the transpose of the above

monum,1,2,3,4,5,6,7,8,9,10,11,12
transactions,572,1625,1664,1727,2076,1495,564,1938,1942,2241,49,257


In [48]:
mo=trDFgrouped.index.get_level_values(0) 	# the month numbers
mo

Int64Index([1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11,
            11, 12, 12],
           dtype='int64', name='monum')

In [49]:
volType=trDFgrouped.index.get_level_values(1)	# vol
volType

Index(['heavy', 'light', 'heavy', 'light', 'heavy', 'light', 'heavy', 'light',
       'heavy', 'light', 'heavy', 'light', 'heavy', 'light', 'heavy', 'light',
       'heavy', 'light', 'heavy', 'light', 'heavy', 'light', 'heavy', 'light'],
      dtype='object', name='vol')

In [50]:
trDFpiv=DataFrame({'month':mo,'vol': volType, 'transactions':trDFgrouped.transactions})		# data as a dict
trDFpiv.head(4)

month    vol  transactions
monum vol                              
1     heavy      1  heavy          5255
      light      1  light           572
2     heavy      2  heavy           761
      light      2  light          1625

In [51]:
trDFpived=trDFpiv.pivot(index='month',columns='vol',values='transactions')
trDFpived

vol,heavy,light
month,,
1,5255,572
2,761,1625
3,1130,1664
4,2327,1727
5,2172,2076
6,2878,1495
7,4440,564
8,1682,1938
9,1921,1942


In [52]:
trDFgrouped.unstack()

transactions      
vol          heavy light
monum                   
1             5255   572
2              761  1625
3             1130  1664
4             2327  1727
5             2172  2076
6             2878  1495
7             4440   564
8             1682  1938
9             1921  1942
10            2109  2241
11            8402    49
12           13168   257

In [53]:
trDFpiv['randy']=np.random.randn(len(trDFpiv))

In [54]:
len(trDFpiv)

24

In [55]:
trDFpived2=trDFpiv.pivot(index='month',columns='vol')
trDFpived2

transactions           randy          
vol          heavy light     heavy     light
month                                       
1             5255   572 -1.067219  1.034217
2              761  1625  0.209018 -0.023390
3             1130  1664  0.487063  0.743045
4             2327  1727 -0.222495  0.218071
5             2172  2076  0.659444  1.174523
6             2878  1495 -0.223234  0.374025
7             4440   564  1.610295 -0.129922
8             1682  1938 -0.812025 -0.031330
9             1921  1942 -0.997889  1.440815
10            2109  2241 -1.151826 -0.127521
11            8402    49 -0.423529  0.490692
12           13168   257  1.255319  0.635785

In [56]:
trDFpiv.drop('randy',axis=1,inplace=True)
trDFpiv

month    vol  transactions
monum vol                              
1     heavy      1  heavy          5255
      light      1  light           572
2     heavy      2  heavy           761
      light      2  light          1625
3     heavy      3  heavy          1130
      light      3  light          1664
4     heavy      4  heavy          2327
      light      4  light          1727
5     heavy      5  heavy          2172
      light      5  light          2076
6     heavy      6  heavy          2878
      light      6  light          1495
7     heavy      7  heavy          4440
      light      7  light           564
8     heavy      8  heavy          1682
      light      8  light          1938
9     heavy      9  heavy          1921
      light      9  light          1942
10    heavy     10  heavy          2109
      light     10  light          2241
11    heavy     11  heavy          8402
      light     11  light            49
12    heavy     12  heavy         13168
      light     12  light           257

In [58]:
trDFpived2

transactions           randy          
vol          heavy light     heavy     light
month                                       
1             5255   572 -1.067219  1.034217
2              761  1625  0.209018 -0.023390
3             1130  1664  0.487063  0.743045
4             2327  1727 -0.222495  0.218071
5             2172  2076  0.659444  1.174523
6             2878  1495 -0.223234  0.374025
7             4440   564  1.610295 -0.129922
8             1682  1938 -0.812025 -0.031330
9             1921  1942 -0.997889  1.440815
10            2109  2241 -1.151826 -0.127521
11            8402    49 -0.423529  0.490692
12           13168   257  1.255319  0.635785

In [59]:
xyzdata=xyzcustnew[['BUYER_STATUS','heavyCat','CHANNEL_ACQUISITION']]
xyzdata.head()

,BUYER_STATUS,heavyCat,CHANNEL_ACQUISITION
0,INACTIVE,regular,IB
1,ACTIVE,heavy,RT
2,ACTIVE,regular,RT
3,INACTIVE,regular,RT
4,ACTIVE,regular,RT


In [60]:
xyzgrouped=xyzdata.groupby(['BUYER_STATUS','heavyCat','CHANNEL_ACQUISITION'])
xyzgrouped

In [61]:
xyzCountData = xyzgrouped.size()	# a MultiIndexed Series of counts [ sum and count do nothing here.******]
xyzCountData

BUYER_STATUS  heavyCat  CHANNEL_ACQUISITION
ACTIVE        regular   CB                      443
                        IB                     1112
                        RT                     7393
              heavy     CB                      356
                        IB                      703
                        RT                     3325
INACTIVE      regular   CB                      691
                        IB                     1249
                        RT                     7056
LAPSED        regular   CB                      372
                        IB                     1111
                        RT                     6368
dtype: int64

In [62]:
print(type(xyzCountData.unstack()))
print(xyzCountData.unstack())
xyzCountData.unstack().unstack()

<class 'pandas.core.frame.DataFrame'>
CHANNEL_ACQUISITION     CB    IB    RT
BUYER_STATUS heavyCat                 
ACTIVE       regular   443  1112  7393
             heavy     356   703  3325
INACTIVE     regular   691  1249  7056
LAPSED       regular   372  1111  6368


CHANNEL_ACQUISITION      CB             IB             RT        
heavyCat            regular  heavy regular  heavy regular   heavy
BUYER_STATUS                                                     
ACTIVE                443.0  356.0  1112.0  703.0  7393.0  3325.0
INACTIVE              691.0    NaN  1249.0    NaN  7056.0     NaN
LAPSED                372.0    NaN  1111.0    NaN  6368.0     NaN

In [63]:
unStackxyz=xyzCountData.unstack()		# what we had just above

In [64]:
unStackxyz.T.stack()		# .T is the transpose 

BUYER_STATUS                  ACTIVE  INACTIVE  LAPSED
CHANNEL_ACQUISITION heavyCat                          
CB                  regular      443     691.0   372.0
                    heavy        356       NaN     NaN
IB                  regular     1112    1249.0  1111.0
                    heavy        703       NaN     NaN
RT                  regular     7393    7056.0  6368.0
                    heavy       3325       NaN     NaN

In [65]:
unStackxyz.T.stack(0).unstack(1)

heavyCat            regular                   heavy                
BUYER_STATUS         ACTIVE INACTIVE LAPSED  ACTIVE INACTIVE LAPSED
CHANNEL_ACQUISITION                                                
CB                      443      691    372   356.0      NaN    NaN
IB                     1112     1249   1111   703.0      NaN    NaN
RT                     7393     7056   6368  3325.0      NaN    NaN

In [66]:
unStackxyz.T.stack(level=['heavyCat','BUYER_STATUS'])

CHANNEL_ACQUISITION  heavyCat  BUYER_STATUS
CB                   regular   ACTIVE           443.0
                               INACTIVE         691.0
                               LAPSED           372.0
                     heavy     ACTIVE           356.0
IB                   regular   ACTIVE          1112.0
                               INACTIVE        1249.0
                               LAPSED          1111.0
                     heavy     ACTIVE           703.0
RT                   regular   ACTIVE          7393.0
                               INACTIVE        7056.0
                               LAPSED          6368.0
                     heavy     ACTIVE          3325.0
dtype: float64

In [67]:
unStackxyz.T.stack(level=['BUYER_STATUS','heavyCat'])

CHANNEL_ACQUISITION  BUYER_STATUS  heavyCat
CB                   ACTIVE        regular      443.0
                                   heavy        356.0
                     INACTIVE      regular      691.0
                     LAPSED        regular      372.0
IB                   ACTIVE        regular     1112.0
                                   heavy        703.0
                     INACTIVE      regular     1249.0
                     LAPSED        regular     1111.0
RT                   ACTIVE        regular     7393.0
                                   heavy       3325.0
                     INACTIVE      regular     7056.0
                     LAPSED        regular     6368.0
dtype: float64

In [68]:
xyzcust=xyzcustnew[['BUYER_STATUS','heavyCat','LTD_SALES']].copy()
xyzcust.head(4)

,BUYER_STATUS,heavyCat,LTD_SALES
0,INACTIVE,regular,90.0
1,ACTIVE,heavy,4227.0
2,ACTIVE,regular,420.0
3,INACTIVE,regular,6552.0


In [69]:
xyzcustm=pd.melt(xyzcust,id_vars=['BUYER_STATUS','heavyCat'],var_name="LTD_SALES")


In [70]:
print(xyzcustm)

      BUYER_STATUS heavyCat  LTD_SALES   value
0         INACTIVE  regular  LTD_SALES    90.0
1           ACTIVE    heavy  LTD_SALES  4227.0
2           ACTIVE  regular  LTD_SALES   420.0
3         INACTIVE  regular  LTD_SALES  6552.0
4           ACTIVE  regular  LTD_SALES   189.0
5           ACTIVE  regular  LTD_SALES  4278.0
6           ACTIVE    heavy  LTD_SALES  1869.0
7           ACTIVE  regular  LTD_SALES    33.0
8         INACTIVE  regular  LTD_SALES   735.0
9         INACTIVE  regular  LTD_SALES   468.0
10          ACTIVE  regular  LTD_SALES   804.0
11          LAPSED  regular  LTD_SALES   219.0
12          ACTIVE    heavy  LTD_SALES  3240.0
13        INACTIVE  regular  LTD_SALES   180.0
14          ACTIVE  regular  LTD_SALES   423.0
15        INACTIVE  regular  LTD_SALES   306.0
16          LAPSED  regular  LTD_SALES  1002.0
17          ACTIVE  regular  LTD_SALES  1155.0
18          ACTIVE  regular  LTD_SALES   612.0
19          ACTIVE  regular  LTD_SALES   633.0
20        INA

In [71]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS','heavyCat'],columns=['CHANNEL_ACQUISITION'])


CHANNEL_ACQUISITION             CB           IB           RT
BUYER_STATUS heavyCat                                       
ACTIVE       regular    205.334086   191.047662   167.993913
             heavy     2397.606742  1251.559033  1158.506165
INACTIVE     regular      0.000000     0.000000     0.000000
LAPSED       regular      0.000000     0.000000     0.000000

In [72]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'])


heavyCat                regular                                heavy  \
CHANNEL_ACQUISITION          CB          IB          RT           CB   
BUYER_STATUS                                                           
ACTIVE               205.334086  191.047662  167.993913  2397.606742   
INACTIVE               0.000000    0.000000    0.000000          NaN   
LAPSED                 0.000000    0.000000    0.000000          NaN   

heavyCat                                       
CHANNEL_ACQUISITION           IB           RT  
BUYER_STATUS                                   
ACTIVE               1251.559033  1158.506165  
INACTIVE                     NaN          NaN  
LAPSED                       NaN          NaN

In [73]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'],aggfunc=np.sum)


heavyCat             regular                          heavy            \
CHANNEL_ACQUISITION       CB        IB         RT        CB        IB   
BUYER_STATUS                                                            
ACTIVE               90963.0  212445.0  1241979.0  853548.0  879846.0   
INACTIVE                 0.0       0.0        0.0       NaN       NaN   
LAPSED                   0.0       0.0        0.0       NaN       NaN   

heavyCat                        
CHANNEL_ACQUISITION         RT  
BUYER_STATUS                    
ACTIVE               3852033.0  
INACTIVE                   NaN  
LAPSED                     NaN

In [74]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'],aggfunc=np.sum,margins=True)


heavyCat             regular                          heavy            \
CHANNEL_ACQUISITION       CB        IB         RT        CB        IB   
BUYER_STATUS                                                            
ACTIVE               90963.0  212445.0  1241979.0  853548.0  879846.0   
INACTIVE                 0.0       0.0        0.0       NaN       NaN   
LAPSED                   0.0       0.0        0.0       NaN       NaN   
All                  90963.0  212445.0  1241979.0  853548.0  879846.0   

heavyCat                              All  
CHANNEL_ACQUISITION         RT             
BUYER_STATUS                               
ACTIVE               3852033.0  7130814.0  
INACTIVE                   NaN        0.0  
LAPSED                     NaN        0.0  
All                  3852033.0  7130814.0

In [75]:
xyzcustnew.head()

,index,ACCTNO,ZIP,ZIP4,LTD_SALES,LTD_TRANSACTIONS,YTD_SALES_2009,YTD_TRANSACTIONS_2009,CHANNEL_ACQUISITION,BUYER_STATUS,ZIP9_SUPERCODE,heavyCat,typeReg,typeHeavy
0,0,WDQQLLDQL,60084,5016,90.0,1,0.0,0,IB,INACTIVE,600845016,regular,1,0
1,1,WQWAYHYLA,60091,1750,4227.0,9,1263.0,3,RT,ACTIVE,600911750,heavy,0,1
2,2,GSHAPLHAW,60067,900,420.0,3,129.0,1,RT,ACTIVE,600670900,regular,1,0
3,3,PGGYDYWAD,60068,3838,6552.0,6,0.0,0,RT,INACTIVE,600683838,regular,1,0
4,4,LWPSGPLLS,60090,3932,189.0,3,72.0,1,RT,ACTIVE,600903932,regular,1,0


In [76]:
xyzGrouper=xyzcustnew.groupby(['BUYER_STATUS','heavyCat'])

In [77]:
xyzGrouper.agg({'YTD_SALES_2009': [np.mean, np.std],'LTD_SALES':[np.mean,np.std]})


YTD_SALES_2009                 LTD_SALES              
                                mean          std         mean           std
BUYER_STATUS heavyCat                                                       
ACTIVE       regular      172.707532   107.584023  1001.845105   1466.075631
             heavy       1274.048130  5434.616517  4096.179745  34210.646330
INACTIVE     regular        0.000000     0.000000   568.014784    850.966479
LAPSED       regular        0.000000     0.000000   841.467329   1374.447756

In [78]:
def coefV(x):                       # a baby CV function that accepts a sequence
    return np.std(x)/np.mean(x)


In [110]:
buyerStats=xyzcustnew[['BUYER_STATUS','LTD_SALES','LTD_TRANSACTIONS']]
buyerGrouper=buyerStats.groupby(['BUYER_STATUS'])
buyerGrouper.agg(coefV)

,LTD_SALES,LTD_TRANSACTIONS
BUYER_STATUS,,
ACTIVE,9.758480,1.153501
INACTIVE,1.498058,0.784441
LAPSED,1.633290,0.987139


In [80]:
def ptiles(x):
    p5=np.percentile(x,5)
    p95=np.percentile(x,95)
    return p5, p95


In [81]:
buyerGrouper.agg([np.mean, ptiles])

LTD_SALES                            LTD_TRANSACTIONS  \
                     mean                     ptiles             mean   
BUYER_STATUS                                                            
ACTIVE        2019.364086  (81.0, 6544.349999999997)         6.935794   
INACTIVE       568.014784             (60.0, 1776.0)         2.263895   
LAPSED         841.467329             (63.0, 2904.0)         3.498280   

                           
                   ptiles  
BUYER_STATUS               
ACTIVE        (1.0, 20.0)  
INACTIVE       (1.0, 6.0)  
LAPSED         (1.0, 9.0)

In [103]:
buyerGrouper.agg([np.mean,ptiles]).loc['ACTIVE','LTD_SALES']

mean                        2019.36
ptiles    (81.0, 6544.349999999997)
Name: ACTIVE, dtype: object

# Requirements :
1. Get the trDFgrouped data starting from the May heavy day counts to the August heavy counts
2. Group xyz customers using BUYER_STATUS, heavyCat, and ZIP, and apply np.sum function on the aggregated data for YTD_SALES_2009 and LTD_SALES columns

In [85]:
# Write your python code that meets the above requirements in this cell
trDFgrouped[(5, 'heavy'):(8,'heavy')]

# your comment and code above are inconsistent in your version. mine above are corrected.  here's your original:
# "The data starting from the March heavy day counts to the July light counts:"
# "trDFgrouped[(3,'light'):(7,'heavy')]"

transactions
monum vol                
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495
7     heavy          4440
      light           564
8     heavy          1682

In [121]:
xyzcustnew[['YTD_SALES_2009','LTD_SALES','BUYER_STATUS', 'heavyCat','ZIP']].groupby(['BUYER_STATUS', 'heavyCat','ZIP']).agg([np.sum])

YTD_SALES_2009  LTD_SALES
                                       sum        sum
BUYER_STATUS heavyCat ZIP                            
ACTIVE       regular  60056        68913.0   332196.0
                      60060        68520.0   339567.0
                      60061        68328.0   400569.0
                      60062       141237.0   762387.0
                      60064         2169.0     9129.0
                      60065         1002.0     2784.0
                      60067       156429.0   922680.0
                      60068       140133.0   802815.0
                      60069        43623.0   280686.0
                      60070        24051.0   134265.0
                      60071         4311.0    20112.0
                      60072         2037.0    14583.0
                      60073        29877.0   143901.0
                      60074        72999.0   349026.0
                      60076        53040.0   252438.0
                      60077        39546.0   183588.0
                      60078         1878.0     7410.0
                      60081        16446.0    76662.0
                      60083        14445.0    81954.0
                      60084        39834.0   243837.0
                      60085        18714.0    88857.0
                      60087        13749.0    59997.0
                      60088         1053.0     2538.0
                      60089       100038.0   481086.0
                      60090        32934.0   153108.0
                      60091       178533.0  1127982.0
                      60093       169671.0  1449606.0
                      60094          357.0      543.0
                      60096         5544.0    34929.0
                      60097         5805.0    29565.0
...                                    ...        ...
LAPSED       regular  60064            0.0     3537.0
                      60065            0.0     7359.0
                      60067            0.0   682167.0
                      60068            0.0   571056.0
                      60069            0.0   134685.0
                      60070            0.0    75333.0
                      60071            0.0    11232.0
                      60072            0.0     2463.0
                      60073            0.0   100932.0
                      60074            0.0   245877.0
                      60076            0.0   207912.0
                      60077            0.0   135801.0
                      60078            0.0     4173.0
                      60079            0.0     2928.0
                      60081            0.0    50397.0
                      60082            0.0      225.0
                      60083            0.0    71463.0
                      60084            0.0   157020.0
                      60085            0.0    60144.0
                      60087            0.0    45030.0
                      60088            0.0     3354.0
                      60089            0.0   407976.0
                      60090            0.0   137544.0
                      60091            0.0   820053.0
                      60093            0.0   955428.0
                      60095            0.0      300.0
                      60096            0.0    17559.0
                      60097            0.0    30564.0
                      60098            0.0   149418.0
                      60192            0.0     4548.0

[132 rows x 2 columns]